In [ ]:
# Common libraries
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
from PIL import Image
import random

# TensorFlow imports
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D,\
                                    GlobalAveragePooling2D, BatchNormalization                                    
from tensorflow.keras import utils, regularizers
from tensorflow.keras.callbacks import EarlyStopping, History
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.optimizers import Adam

# Setting a random seed
np.random.seed(42)

In [ ]:
# Creating targets
TARGETS = ['Normal', 'Cancerous']

# Setting path to data directory
train_dir = './cnn_train'
test_dir = './cnn_test'

In [ ]:
# Creating an empty list to be populated by the image array and class
training_data = []
def create_training_data():
    for target in TARGETS:
        # Join the directory and target
        path = os.path.join(train_dir, target)
        # Index the target 0-normal, 1-cancerous
        class_num = TARGETS.index(target)
        for img in os.listdir(path):
            # Openning an image by joining the path with the image file name
            im = Image.open(os.path.join(path, img))
            # Create a np.array of the image
            img_array = np.array(im)
            # Append the array and class index to the empty list
            training_data.append([img_array, class_num])
                            
create_training_data()

In [ ]:
# Creating an empty list to be populated by the image array and class
testing_data = []
def create_testing_data():
    for target in TARGETS:
        # Join the directory and target
        path = os.path.join(test_dir, target)
        # Index the target 0-normal, 1-cancerous
        class_num = TARGETS.index(target)
        for img in os.listdir(path):
            # Openning an image by joining the path with the image file name
            im = Image.open(os.path.join(path, img))
            # Create a np.array of the image
            img_array = np.array(im)
            # Append the array and class index to the empty list
            testing_data.append([img_array, class_num])
            
create_testing_data()

In [ ]:
X_train = []
y_train = []

# Creating X_train - y_train
for features, label in training_data:
    X_train.append(features)
    y_train.append(label)
  
 # Reshape the x_train array
X_train = np.array(X_train).reshape(-1, 96, 96, 3)

In [ ]:
X_test = []
y_test = []

# Creating X_test, y_test
for features, label in testing_data:
    X_test.append(features)
    y_test.append(label)
    
 # Reshape the x_test array
X_test = np.array(X_test).reshape(-1, 96, 96, 3)

In [1]:
model = Sequential()

model.add(ResNet50(
        include_top = False,
        weights = "imagenet",
        input_shape = (96, 96, 3)
))

model.add(Dropout(0.5))

model.add(GlobalAveragePooling2D())

model.add(Dense(
        128, 
        activation = "relu",
        kernel_regularizer = regularizers.l2(0.5)

))

model.add(BatchNormalization())

model.add(Dense(
        1,
        activation = "sigmoid",
        kernel_regularizer = regularizers.l2(0.5)

))


es = EarlyStopping(
    monitor = "val_loss",
    min_delta = 0,
    patience = 5
)

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['acc']
)

/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/Caskroom/miniconda/base/lib/python3

In [ ]:
history = model.fit(
        X_train,
        y_train,
        batch_size = 256,
        validation_data = (X_test, y_test),
        epochs = 10,
        callbacks = [es],
        verbose = 1,
        shuffle = True
)